# Object Detection Performance Check

# Imports

In [ ]:
from __future__ import print_function

# from matplotlib import pyplot as plt
# from matplotlib.pyplot import imshow
import numpy as np
import os
from PIL import Image
from PIL import ImageDraw
from PIL import ImageColor
from PIL import ImageFont
import tensorflow as tf
import xml.etree.ElementTree as ET
from scipy.stats import norm
from IPython.display import display

print("TensorFlow version:", tf.VERSION)

# Model preparation

Export a trained model. Set class info. Set path to test images. 

In [ ]:
# What model to load
# MODEL_NAME = 'training/model1/trained_model'
MODEL_NAME = 'training/model2/trained_model'
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data/', 'label_map_sdc.pbtxt')

In [ ]:
# shoud match with the order in label_map_sdc.pbtxt
CLASSNAME_LIST = ['Green', 'Red', 'Yellow'] # list of class name 
COLOR_LIST = ['lawngreen', 'red', 'yellow'] # list of color to be used for visual purpose below 

## Load Frozen Tensorflow Model into Memory

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Helper Code

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def filter_boxes(min_score, boxes, scores, classes):
    """Return boxes with a confidence >= `min_score`"""
    n = len(classes)
    idxs = []
    for i in range(n):
        if scores[i] >= min_score:
            idxs.append(i)
    
    filtered_boxes = boxes[idxs, ...]
    filtered_scores = scores[idxs, ...]
    filtered_classes = classes[idxs, ...]
    return filtered_boxes, filtered_scores, filtered_classes

def to_image_coords(boxes, height, width):
    """
    The original box coordinate output is normalized, i.e [0, 1].
    
    This converts it back to the original coordinate based on the image
    size.
    """
    box_coords = np.zeros_like(boxes)
    box_coords[:, 0] = boxes[:, 0] * height
    box_coords[:, 1] = boxes[:, 1] * width
    box_coords[:, 2] = boxes[:, 2] * height
    box_coords[:, 3] = boxes[:, 3] * width
    
    return box_coords

def draw_boxes(image, boxes, classes, scores, thickness=4):
    """Draw bounding boxes on the image"""
    image_draw = image.copy()
    draw = ImageDraw.Draw(image_draw)
    for i in range(len(boxes)):
        bot, left, top, right = boxes[i, ...]
        class_id = int(classes[i])
        color = COLOR_LIST[class_id-1]
        cls_name = CLASSNAME_LIST[class_id-1]
        percent = str(round(scores[i] * 100, 1))
        txt_display = cls_name + ": " + percent + "%"
        # print(class_id, cls_name, color, txt_display)
        # draw.rectangle([(left, top-15), (left+80, top-thickness)], fill= color)
        draw.rectangle([(left-2, bot-15), (left+80, bot)], fill= color)
        draw.line([(left, top), (left, bot), (right, bot), (right, top), (left, top)], width=thickness, fill=color)
        draw.text((left, bot-15), txt_display, fill="black")
    return image_draw

def load_graph(graph_file):
    """Loads a frozen inference graph"""
    graph = tf.Graph()
    with graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(graph_file, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    return graph

# Detection

In [ ]:
detection_graph = load_graph(PATH_TO_FROZEN_GRAPH)

# The input placeholder for the image.
# `get_tensor_by_name` returns the Tensor with the associated name in the Graph.
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Each box represents a part of the image where a particular object was detected.
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represent how level of confidence for each of the objects.
# Score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')

# The classification of the object (integer id).
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

In [ ]:
# path to test image directory 
TEST_IMAGES_DIR = 'data/v3_sim_data_val'

# path to detected images
DETECTED_IMAGES_DIR = TEST_IMAGES_DIR + '/detected_images'
if not os.path.exists(DETECTED_IMAGES_DIR):
        os.makedirs(DETECTED_IMAGES_DIR)

# Read source filename_list.txt
test_files = open(TEST_IMAGES_DIR + '/filename_list.txt', 'r').read().split('\n')
if '' in test_files:
    test_files.remove('')

print("Using {} test images found in {}/images".format(len(test_files), TEST_IMAGES_DIR))

In [ ]:
minor_errors = 0
major_errors = 0

for i, filename in enumerate(test_files):

    image_path = TEST_IMAGES_DIR + '/images/' + filename + '.jpg'
    annotation_path = TEST_IMAGES_DIR + '/annotations/' + filename + '.xml'

    # Load the sample image.
    image = Image.open(image_path)
    image_np = np.expand_dims(np.asarray(image, dtype=np.uint8), 0)
    print("[{}] Processing {}".format(i+1, image_path))

    ground_truth = {
        'Red': 0,
        'Green': 0,
        'Yellow': 0
    }

    detection = {
        'Red': 0,
        'Green': 0,
        'Yellow': 0
    }

    # reading the annotation file
    annotation = ET.parse(annotation_path).getroot()
    for tl_object in annotation.findall('object'):
        tl_name = tl_object.find('name')
        ground_truth[tl_name.text] += 1

    with tf.Session(graph=detection_graph) as sess:                

        # Actual detection.
        (boxes, scores, classes) = sess.run([detection_boxes, detection_scores, detection_classes], 
                                            feed_dict={image_tensor: image_np})
        # Remove unnecessary dimensions
        boxes = np.squeeze(boxes)
        scores = np.squeeze(scores)
        classes = np.squeeze(classes)

        confidence_cutoff = 0.8
        # Filter boxes with a confidence score less than `confidence_cutoff`
        boxes, scores, classes = filter_boxes(confidence_cutoff, boxes, scores, classes)

        # collecting detection data
        for class_id in classes:
            detected_class = CLASSNAME_LIST[int(class_id) - 1]
            detection[detected_class] += 1

        # checking against the ground truth
        display_image = False
        suffix = None
        for light in ground_truth:

            if ground_truth[light] != detection[light]:

                display_image = True
                if ground_truth[light] == 0 or detection[light] == 0:
                    major_errors += 1
                    suffix = "major"
                    print("   *** Error: there's a significant difference for {} ".format(light) +
                          "(ground truth: {}, detection: {})".format(ground_truth[light], detection[light]))

                else:
                    minor_errors += 1
                    if suffix != "major":
                        suffix = "minor"
                    print("   * Warning: there's a difference for {} but it's still OK ".format(light) +
                          "(ground truth: {}, detection: {})".format(ground_truth[light], detection[light]))

        if display_image:
            # The current box coordinates are normalized to a range between 0 and 1.
            # This converts the coordinates actual location on the image.
            width, height = image.size
            box_coords = to_image_coords(boxes, height, width)

            # Each class with be represented by a differently colored box
            image_draw = draw_boxes(image, box_coords, classes, scores)

            display(image_draw)

            save_image_path = DETECTED_IMAGES_DIR + '/' + filename + "_detected_" + suffix + ".jpg"
            image_draw.save(save_image_path)

In [ ]:
# -------------------------------------------- data/v2_sim_data_val
# # Displaying some statistics
# all_count = len(test_files)
# passed = all_count - major_errors
# print("Total image count:", all_count)
# print("Minor errors: {} ({:.2f}%)".format(minor_errors, (100.0 * minor_errors / all_count)))
# print("Major errors: {} ({:.2f}%)".format(major_errors, (100.0 * major_errors / all_count)))
# print("Passed: {} ({:.2f}%))".format(passed, (100.0 * passed / all_count)))

# Total image count: 232
# Minor errors: 11 (4.74%)
# Major errors: 1 (0.43%)
# Passed: 231 (99.57%))

In [ ]:
# -------------------------------------------- data/v3_sim_data_val
# Displaying some statistics
all_count = len(test_files)
passed = all_count - major_errors
print("Total image count:", all_count)
print("Minor errors: {} ({:.2f}%)".format(minor_errors, (100.0 * minor_errors / all_count)))
print("Major errors: {} ({:.2f}%)".format(major_errors, (100.0 * major_errors / all_count)))
print("Passed: {} ({:.2f}%))".format(passed, (100.0 * passed / all_count)))